In [1]:
import gradio as gr
import numpy as np
from web3 import Web3, HTTPProvider
from pyevmasm import disassemble_hex
from dotenv import load_dotenv
import os
import pickle
load_dotenv()
INFURA_URL=os.getenv('INFURA_URL') 

/home/amitai/miniconda3/envs/ml2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
w3 = Web3(HTTPProvider(INFURA_URL))

In [3]:
def predict(address):
    # Load models and tfidf vectorizer
    with open('./weights/logistic_model.pkl', 'rb') as f:
        logistic_model = pickle.load(f)
    with open('./weights/rf_model.pkl', 'rb') as f:
        rf_model = pickle.load(f)
    with open('./weights/xgb_model.pkl', 'rb') as f:
        xgb_model = pickle.load(f)
    with open('./weights/tfidf.pkl', 'rb') as f:
        tfidf = pickle.load(f)
    
    def get_opcodes(address):
        check_address= w3.to_checksum_address(address)
        bytecode = w3.eth.get_code(check_address).hex()[2:]
        instructions = disassemble_hex(bytecode)
        instructions = instructions.split('\n')
        return ' '.join(instr.split(' ')[0] for instr in instructions)

    opcodes = get_opcodes(address)
    # Vectorize
    opcodes_vectorized = tfidf.transform([opcodes])

    # Make predictions
    logistic_prediction = logistic_model.predict_proba(opcodes_vectorized)[0][1].item()
    rf_prediction = rf_model.predict_proba(opcodes_vectorized)[0][1].item()
    xgb_prediction = xgb_model.predict_proba(opcodes_vectorized)[0][1].item()
    
    # Create classification based on the prediction
    logistic_classification = 'valid' if logistic_prediction < 0.5 else 'malicious'
    rf_classification = 'valid' if rf_prediction < 0.5 else 'malicious'
    xgb_classification = 'valid' if xgb_prediction < 0.5 else 'malicious'

    return {
        'Logistic Regression': {
            'prediction': logistic_prediction, 
            'classification': logistic_classification
        }, 
        'Random Forest': {
            'prediction': rf_prediction, 
            'classification': rf_classification
        }, 
        'XGBoost': {
            'prediction': xgb_prediction, 
            'classification': xgb_classification
        }
    }

In [4]:
iface = gr.Interface(fn=predict, inputs='text', outputs='json', title='Ethereum Contract Classification', examples=[
    ['0x6B3595068778DD592e39A122f4f5a5cF09C90fE2'],  # Valid contract address example
    ['0xc00e94Cb662C3520282E6f5717214004A7f26888'], # Valid contract address example
    ['0x1e891e6c7ea7a7c32d4b9643b90b8a9fa313c77f'], # Malicious contract address example
    ['0x55404BdE53423030F7f50dE86B9C4167165cF61E']   # Malicious contract address example
])
iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
